In [34]:
# KMeans Clustering
import numpy as np
from typing import List, Tuple

In [37]:
class KMeansScratch:
    def __init__(self, clusters, iterations, tolerance: float = 1e-4, random_state: int = None):
        self.clusters = clusters
        self.iterations = iterations
        self.tolerance = tolerance
        self.random_state = random_state
        self.centroids: np.ndarray | None = None # Type hint for centroids
        self.min_dist: np.ndarray | None = None # Type hint for cluster labels
    
    def _init_centroids(self, points, initial_centroids):
        # if initial centroids provided -> convert to numpy arrays else randomly select len(cluster) points from datasets
        if initial_centroids:
            if len(initial_centroids) != self.clusters:
                raise ValueError("Number of initial centroids do not match with cluster size")
            self.centroids = np.array(initial_centroids, dtype=np.float64)
        else:
            if self.random_state is not None:
                np.random.seed(self.random_state)
            random_idx = np.random.permutation(points.shape[0])[:self.clusters]
            self.centroids = points[random_idx].astype(np.float64)

    def _assign_clusters(self, points):
        # assigns each data pt to nearest centroid
        squared_dist = np.sum((points[:, np.newaxis, :] - self.centroids)**2, axis=2)
        min_dist = np.argmin(squared_dist, axis=1)
        return min_dist
    
    def _update_centroids(self, points, min_dist):
        # recalculates centroids based on mean
        new_centroids = np.zeros_like(self.centroids, dtype=np.float64)
        for i in range(self.clusters):
            cluster_pts = points[min_dist == i]
            if cluster_pts.shape[0] > 0:
                new_centroids[i] = np.mean(cluster_pts, axis = 0)
            else:
                # if cluster empty, keep centroid in it's curr posn
                new_centroids[i] = self.centroids[i] 
        new_centroids = np.round(new_centroids, 4)
        return new_centroids
    
    def fit(self, points, initial_centroids: List[Tuple] | None = None):
        # performs k means on inp data
        points_arr = np.array(points, dtype=np.float64)
        self._init_centroids(points_arr, initial_centroids)
        for iteration in range(self.iterations):
            # to check for convergence later
            old_centroids = self.centroids.copy() 
            # assign
            self.min_dist = self._assign_clusters(points_arr)
            # update
            self.centroids = self._update_centroids(points_arr, self.min_dist)

            # comparision for convergence
            if np.allclose(old_centroids, self.centroids, atol=self.tolerance): 
                print(f"Converged after {iteration + 1} iterations")
                break
        if iteration == self.iterations - 1 and not np.allclose(old_centroids, self.centroids, atol=self.tol):
             print(f"Warning: Did not converge within {self.iterations} iterations.")
    
    def get_centroids(self):
        # returns final centroid coordinates as list of tuples
        if self.centroids is None:
            raise RuntimeError("Doesn't work")
        return [tuple(float(x) for x in centroid) for centroid in self.centroids]
    
    def get_minDist(self):
        # returns cluster labels/min dist for each data pt
        return self.min_dist


In [41]:
def main():
    # 2D
    points_2d = [(1, 2), (1, 4), (1, 0), (10, 2), (10, 4), (10, 0)]

    print("-" * 20)
    k_2d = 2 
    initial_centroids_2d = [(1, 1), (10, 1)]
    max_iterations = 10
    kmeans_2d = KMeansScratch(clusters=k_2d, iterations=max_iterations, tolerance=1e-4, random_state=42)
    kmeans_2d.fit(points=points_2d, initial_centroids=initial_centroids_2d)
    print("Example 1 (2D): Final Centroids =", kmeans_2d.get_centroids())
    # [(1.0, 2.0), (10.0, 2.0)]

    print("-" * 20)
    # 3D
    points_3d = [(0, 0, 0), (2, 2, 2), (1, 1, 1), (9, 10, 9), (10, 11, 10), (12, 11, 12)]
    k_3d = 2
    initial_centroids_3d = [(1, 1, 1), (10, 10, 10)]
    kmeans_3d = KMeansScratch(clusters=k_3d, iterations=max_iterations, tolerance=1e-4, random_state=42)
    kmeans_3d.fit(points=points_3d, initial_centroids=initial_centroids_3d)
    print("Example 2 (3D): Final Centroids =", kmeans_3d.get_centroids())
    # [(1.0, 1.0, 1.0), (10.3333, 10.6667, 10.3333)]

    print("-" * 20)
    print("Example 3: Random Initialization (2D)")
    points_rand = np.random.rand(50, 2) * 10 # 50 random 2D points
    # Fix 7: Provide all arguments to __init__ and use correct parameter names
    kmeans_rand = KMeansScratch(clusters=3, iterations=100, tolerance=1e-4, random_state=42)
    kmeans_rand.fit(points_rand) # initial_centroids is None by default
    print("Example 3 (Random Init): Final Centroids =", kmeans_rand.get_centroids())
main()

--------------------
Converged after 2 iterations
Example 1 (2D): Final Centroids = [(1.0, 2.0), (10.0, 2.0)]
--------------------
Converged after 2 iterations
Example 2 (3D): Final Centroids = [(1.0, 1.0, 1.0), (10.3333, 10.6667, 10.3333)]
--------------------
Example 3: Random Initialization (2D)
Converged after 6 iterations
Example 3 (Random Init): Final Centroids = [(7.4888, 5.8186), (3.185, 2.0094), (2.7172, 7.7442)]
